## Import Libraries

In [1]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import hashlib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import cv2
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from flask import Flask, render_template, request, jsonify, send_from_directory
from keras.models import load_model
from keras.models import save_model
import io
from tensorflow.keras.models import load_model
from werkzeug.utils import secure_filename
from threading import Thread

## Define paths

In [2]:
real_images_path = 'dataset/train/REAL'
fake_images_path = 'dataset/train/FAKE'
real_test_path = 'dataset/test/REAL'
fake_test_path = 'dataset/test/FAKE'

## Load Images from Folders

In [3]:
def load_images_from_folder(folder, label):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            img = Image.open(img_path)
            img = img.resize((128, 128))
            img = np.array(img)
            images.append([img, label])
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
    return images

### Convert to DataFrame

####  1. Train data

In [4]:
real_images = load_images_from_folder(real_images_path, label=0)  # Label 0 for real
fake_images = load_images_from_folder(fake_images_path, label=1)  # Label 1 for fake

# Combine fake and real
all_images = real_images + fake_images

# Convert to df
df = pd.DataFrame(all_images, columns=['image', 'label'])

print(df.head())

                                               image  label
0  [[[174, 175, 185], [173, 173, 184], [169, 169,...      0
1  [[[167, 179, 191], [168, 180, 192], [168, 180,...      0
2  [[[152, 197, 104], [149, 195, 101], [143, 188,...      0
3  [[[131, 126, 122], [130, 125, 121], [128, 123,...      0
4  [[[37, 35, 11], [37, 35, 11], [37, 35, 11], [3...      0


In [5]:
df.tail(10)

,image,label
99990,"[[[235, 233, 234], [235, 233, 234], [235, 233,...",1
99991,"[[[17, 16, 24], [16, 15, 23], [16, 15, 22], [1...",1
99992,"[[[51, 64, 22], [52, 64, 23], [54, 66, 25], [5...",1
99993,"[[[114, 72, 47], [116, 74, 49], [120, 78, 53],...",1
99994,"[[[11, 0, 15], [11, 0, 14], [11, 0, 14], [11, ...",1
99995,"[[[0, 0, 0], [0, 0, 0], [7, 7, 5], [21, 21, 19...",1
99996,"[[[6, 16, 0], [6, 16, 0], [6, 16, 0], [6, 16, ...",1
99997,"[[[81, 79, 80], [81, 79, 80], [81, 79, 80], [8...",1
99998,"[[[33, 42, 0], [35, 42, 0], [35, 42, 0], [36, ...",1
99999,"[[[164, 162, 163], [164, 162, 163], [164, 162,...",1
